In [1]:
import pandas as pd
import numpy as np

import pyomo.environ as pyo

import gurobipy as gp
from gurobipy import GRB

import openpyxl

from icecream import ic


In [2]:
file_path = "Railway services-2024.xlsx"

try:
    df = pd.read_excel(io=file_path, engine="openpyxl")
    ic(df.head())

except FileNotFoundError:
    print(f"File '{file_path}' not found. Please check the file path.")
    

ic| df.head():    Trip Departure Time Arrival Time From To  Demand(μ)  Demand(σ)  Line
               0     1       07:00:00     07:46:00    M  A        327         64   100
               1     2       07:00:00     07:57:00    H  M        936        308   800
               2     3       07:02:00     07:22:00    M  B        461        252   200
               3     4       07:04:00     08:03:00    M  J        428         10  1000
               4     5       07:06:00     08:13:00    M  F        449        124   600


In [3]:
# Data cleaning/ enhancing

# Removing unneccesary spaces from 'From' and 'To' columns. 
df['From'] = df['From'].str.strip()
df['To'] = df['To'].str.strip()

# Adding an indicator when the trip is located on line 400
indicator_line_400 = np.where(df['Line'] == 400.0, 1, 0)
df["line_400"] = indicator_line_400

ic(df)

ic| df:      Trip Departure Time Arrival Time From To  Demand(μ)  Demand(σ)  Line  \
        0       1       07:00:00     07:46:00    M  A        327         64   100   
        1       2       07:00:00     07:57:00    H  M        936        308   800   
        2       3       07:02:00     07:22:00    M  B        461        252   200   
        3       4       07:04:00     08:03:00    M  J        428         10  1000   
        4       5       07:06:00     08:13:00    M  F        449        124   600   
        ..    ...            ...          ...  ... ..        ...        ...   ...   
        195   196       08:57:00     09:30:00    D  M        384        314   400   
        196   197       08:58:00     09:26:00    M  D        395        366   400   
        197   198       08:58:00     09:33:00    M  E        331        436   500   
        198   199       08:59:00     10:02:00    F  M        795         12   600   
        199   200       08:59:00     09:45:00    K  M       1034 

,Trip,Departure Time,Arrival Time,From,To,Demand(μ),Demand(σ),Line,line_400
0,1,07:00:00,07:46:00,M,A,327,64,100,0
1,2,07:00:00,07:57:00,H,M,936,308,800,0
2,3,07:02:00,07:22:00,M,B,461,252,200,0
3,4,07:04:00,08:03:00,M,J,428,10,1000,0
4,5,07:06:00,08:13:00,M,F,449,124,600,0
...,...,...,...,...,...,...,...,...,...
195,196,08:57:00,09:30:00,D,M,384,314,400,1
196,197,08:58:00,09:26:00,M,D,395,366,400,1
197,198,08:58:00,09:33:00,M,E,331,436,500,0
198,199,08:59:00,10:02:00,F,M,795,12,600,0


In [14]:
### CREATING THE MODEL

# Creation of the Model:
model = pyo.ConcreteModel()

# Creation of sets:

train_type = [
    "OC", 
    "OH"
    ]
cost_train_type = {
    "OC": 260000,
    "OH": 210000
}
length_train_type = {
    "OC": 100, 
    "OH": 70
    }
capacity_train_type = {
    "OC": 620, 
    "OH": 420
    }
cross_section = set(
    df['From'].values + df['To'].values
    )

# Cross section based
cross_sections_using_line400 = set(
    df.loc[df['line_400'] == 1, 'From'].values + df.loc[df['line_400'] == 1, 'To'].values
)
# OR Trip based (Comment out for usage of section based version)
cross_sections_using_line400 = (df.loc[df['line_400'] == 1]).index.to_list()

# Regular sets
model.trips = pyo.Set(
    initialize = df['Trip']
    )
#model.trips.pprint()

model.train_type = pyo.Set(
    initialize = train_type
    )
#model.train_type.pprint()

#FIXME: Returns the warning about unordered data. 
model.cross_section = pyo.Set(
    initialize = cross_section
    )
model.cross_section.pprint()

# Index sets
model.index_set_allocation = pyo.Set(
    initialize = model.trips * model.train_type #FIXME: Should this be model.trips or model.cross_section, because every trip has different values for p_c
    )
#model.index_set_allocation.pprint()

# Creating the Parameters. 
model.cost_train_type = pyo.Param(
    model.train_type, 
    initialize = cost_train_type
)
#model.cost_train_type.pprint()

model.length_train_type = pyo.Param(
    model.train_type, 
    initialize = length_train_type
    )
#model.length_train_type.pprint()

model.capacity_train_type = pyo.Param(
    model.train_type, 
    initialize = capacity_train_type
    )
#model.capacity_train_type.pprint()

model.cross_section_using_line400 = pyo.Param(
    model.cross_section,  # Assuming model.cross_section contains all possible cross sections
    initialize={cross_sec: 1 if cross_sec in cross_sections_using_line400 else 0 for cross_sec in model.cross_section}
)
model.cross_section_using_line400.pprint()

ic(cross_sections_using_line400)
model.cross_section.pprint()
for cross_sec in model.cross_section:
    if cross_sec in cross_sections_using_line400:
        ic(cross_sec)

print('succes')
model.passengers_per_trip = pyo.Param(
    model.trips, 
    initialize = df['Demand(μ)']
)

# Creation of the Variables

model.allocation_train_numbers = pyo.Var(
    model.index_set_allocation, 
    domain = pyo.NonNegativeIntegers, 
    name = 'train_allocation', 
    doc = 'The number of trains of a certain type allocated to a cross-section'
    )


#model.allocation_train_numbers.pprint()
#model.passengers_per_trip.pprint()


data source (type: set).  This WILL potentially lead to nondeterministic
behavior in Pyomo


ic| cross_sections_using_line400: [5,
                                   7,
                                   24,
                                   25,
                                   40,
                                   41,
                                   56,
                                   59,
                                   77,
                                   78,
                                   91,
                                   92,
                                   109,
                                   111,
                                   128,
                                   129,
                                   144,
                                   145,
                                   160,
                                   162,
                                   181,
                                   182,
                                   195,
                                   196]


cross_section : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :   24 : {'MD', 'GM', 'HM', 'MH', 'JM', 'MJ', 'DM', 'CM', 'BM', 'LM', 'ME', 'AM', 'MK', 'ML', 'MB', 'MF', 'EM', 'MC', 'FM', 'MI', 'MA', 'KM', 'MG', 'IM'}
cross_section_using_line400 : Size=24, Index=cross_section, Domain=Any, Default=None, Mutable=False
    Key : Value
     AM :     0
     BM :     0
     CM :     0
     DM :     0
     EM :     0
     FM :     0
     GM :     0
     HM :     0
     IM :     0
     JM :     0
     KM :     0
     LM :     0
     MA :     0
     MB :     0
     MC :     0
     MD :     0
     ME :     0
     MF :     0
     MG :     0
     MH :     0
     MI :     0
     MJ :     0
     MK :     0
     ML :     0
cross_section : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :   24 : {'MD', 'GM', 'HM', 'MH', 'JM', 'MJ', 'DM', 'CM', 'BM', 'LM', 'ME', 'AM', 'MK', 'ML', 'MB', 

KeyError: "Index '0' is not valid for indexed component 'passengers_per_trip'"

In [5]:
# Creation of constraints

#FIXME: Should be itterating over trips instead of the set of unique from and to combinations, because every combination has a different value for p. 
#FIXME: Use either trips or cross_section
# Rule for the combined train length
def rule_maximum_length(
        m, 
        cross_section_OR_trips
        ):
    combined_train_length = sum(
        m.length_train_type[(train_type)] * m.allocation_train_numbers[(cross_section_OR_trips, train_type)]
        for train_type in m.train_type
    )
    return (
        0, 
        combined_train_length,
        300 - 100 * m.cross_section_using_line400[(cross_section_OR_trips)]
    )
model.constr_maximum_length = pyo.Constraint(
    model.trips, 
    rule = rule_maximum_length
    ) #FIXME: trips or cross section.


# Rule for the required combination of the number of trains per train type based on the expected number of passengers. 
def rule_passenger_limit(
        m, 
        cross_section_OR_trips
        ):
    used_capacity = sum(
        m.capacity_train_type[(train_type)] * m.allocation_train_numbers[(cross_section_OR_trips, train_type)]
        for train_type in m.train_type
    )
    return (0, m.passengers_per_trip[(cross_section_OR_trips)], used_capacity)
model.contr_passenger_limit = pyo.Constraint(
    model.trips, 
    rule = rule_passenger_limit
    ) #FIXME: trips or cross section



def rule_difference_between_number_of_train_types1(m):
    return (
        0,
        0.75 * sum(
            m.allocation_train_numbers[(trip, train_type)] 
            for trip in m.trips 
            for train_type in m.train_type if train_type == "OC"
        ),
        sum(
            m.allocation_train_numbers[(trip, train_type)] 
            for trip in m.trips 
            for train_type in m.train_type if train_type == "OH"
        )
    )
model.constr_difference_between_number_of_train_types1 = pyo.Constraint(
    rule=rule_difference_between_number_of_train_types1
)

def rule_difference_between_number_of_train_types2(m):
    return (
        0,
        0.75 * sum(
            m.allocation_train_numbers[(trip, train_type)] 
            for trip in m.trips 
            for train_type in m.train_type if train_type == "OH"
        ),
        sum(
            m.allocation_train_numbers[(trip, train_type)] 
            for trip in m.trips 
            for train_type in m.train_type if train_type == "OC"
        )
    )
model.constr_difference_between_number_of_train_types2 = pyo.Constraint(
    rule=rule_difference_between_number_of_train_types2
)

def obj_minimize_total_cost(m):
    total_cost = sum(
        m.cost_train_type[(train_type)] * sum(
            m.allocation_train_numbers[(trip, train_type)] 
            for trip in m.trips
        )
        for train_type in m.train_type
    )

    return total_cost
model.objective = pyo.Objective(
    rule=obj_minimize_total_cost,
    sense=pyo.minimize
)


ERROR: Rule failed when generating expression for Constraint
constr_maximum_length with index 1: KeyError: "Index '1' is not valid for
indexed component 'cross_section_using_line400'"
ERROR: Constructing component 'constr_maximum_length' from data=None failed:
KeyError: "Index '1' is not valid for indexed component
'cross_section_using_line400'"


KeyError: "Index '1' is not valid for indexed component 'cross_section_using_line400'"

In [ ]:
from itertools import product

# Define the lengths of each train type
OH_length = 100
OC_length = 70

# Define the maximum length of a train
max_train_length = 300

# Generate all possible combinations of OH and OC trains
all_combinations = {}

for num_OH in range(max_train_length // OH_length + 1):
    for num_OC in range(max_train_length // OC_length + 1):

        length = num_OH * OH_length + num_OC * OC_length
        if length <= max_train_length:
            combination = ()
            i = 0
            while i <= num_OH:
                if not i == 0:
                    combination += ('OH',)
                i += 1
            
            i = 0
            while i <= num_OC:
                if not i == 0:
                    combination += ('OC',)
                i += 1

            all_combinations[combination] = length

ic(all_combinations)

ic| all_combinations: {(): 0,
                       ('OC',): 70,
                       ('OC', 'OC'): 140,
                       ('OC', 'OC', 'OC'): 210,
                       ('OC', 'OC', 'OC', 'OC'): 280,
                       ('OH',): 100,
                       ('OH', 'OC'): 170,
                       ('OH', 'OC', 'OC'): 240,
                       ('OH', 'OH'): 200,
                       ('OH', 'OH', 'OC'): 270,
                       ('OH', 'OH', 'OH'): 300}


{(): 0,
 ('OC',): 70,
 ('OC', 'OC'): 140,
 ('OC', 'OC', 'OC'): 210,
 ('OC', 'OC', 'OC', 'OC'): 280,
 ('OH',): 100,
 ('OH', 'OC'): 170,
 ('OH', 'OC', 'OC'): 240,
 ('OH', 'OH'): 200,
 ('OH', 'OH', 'OC'): 270,
 ('OH', 'OH', 'OH'): 300}

In [ ]:
ic((df.loc[df['line_400'] == 1]).index.to_list())

ic| (df.loc[df['line_400'] == 1]).index: Index([  5,   7,  24,  25,  40,  41,  56,  59,  77,  78,  91,  92, 109, 111,
                                                128, 129, 144, 145, 160, 162, 181, 182, 195, 196],
                                               dtype='int64')


Index([  5,   7,  24,  25,  40,  41,  56,  59,  77,  78,  91,  92, 109, 111,
       128, 129, 144, 145, 160, 162, 181, 182, 195, 196],
      dtype='int64')